In [2]:
import numpy as np
import pandas as pd
import redis

## Connect to redis client

In [5]:
# redis-12955.crce179.ap-south-1-1.ec2.redns.redis-cloud.com:12955
# 0AXzSpM3wa8utdJhuvvpRF8Iwc4TNVT3

hostname = 'redis-12955.crce179.ap-south-1-1.ec2.redns.redis-cloud.com'
portnumber = 12955
password = '0AXzSpM3wa8utdJhuvvpRF8Iwc4TNVT3'

In [6]:
r = redis.StrictRedis(host=hostname, port=portnumber, password=password)

In [7]:
r.ping()

True

In [17]:
r.keys()

[b'academy:register']

## Load the data (Numpy zip)

In [9]:
np_file = np.load('dataframe_students_teacher.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

In [10]:
df = pd.DataFrame(x_values, columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43231916427612305, 1.0750129699707032, -0.1..."
1,Barack Obama,Teacher,"[0.9486202239990235, -0.5583913326263428, 0.09..."
2,Chris Evans,Student,"[0.7201780319213867, -1.4683103561401367, -0.7..."
3,Morgan Freeman,Teacher,"[0.48572502136230467, -0.6195638179779053, -0...."
4,Scarlett Johansson,Student,"[0.19124267101287842, -0.4888328552246094, -1...."


In [11]:
df['name_role'] = df['Name'] + '@' + df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43231916427612305, 1.0750129699707032, -0.1...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.9486202239990235, -0.5583913326263428, 0.09...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7201780319213867, -1.4683103561401367, -0.7...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48572502136230467, -0.6195638179779053, -0....",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19124267101287842, -0.4888328552246094, -1....",Scarlett Johansson@Student


In [15]:
records = df[['name_role', 'Facial_Features']].to_dict(orient='records')

5

In [16]:
records[0]

{'name_role': 'Angelina Jolie@Student',
 'Facial_Features': array([ 4.32319164e-01,  1.07501297e+00, -1.28418267e-01, -1.83340302e+00,
        -7.27674103e-01, -1.23460455e+00, -2.34214354e-01, -3.72725034e-01,
        -6.96603966e-01, -3.52733588e-01, -1.86551499e-01, -8.56555843e-01,
        -7.25685883e-01,  1.59820299e+00,  1.89326990e-01, -8.70843172e-02,
        -8.61188591e-02,  9.61314678e-02, -4.32047319e-01,  5.77220380e-02,
         1.06157017e-01, -1.08427849e+00, -3.68412757e-01, -3.46532869e-01,
         5.47845697e-01, -3.21464109e-01,  1.65759449e+00,  3.66927099e-01,
         9.40627193e-01,  1.18184509e+00,  4.02608442e-01,  5.15139008e-01,
         2.07586975e+00, -1.63650761e+00, -1.61403584e-01, -9.76156831e-02,
        -7.65433455e-01,  7.55847740e-01, -1.70595284e+00,  2.02059603e-01,
        -1.64418964e+00,  5.47350597e-01,  2.28774929e-01, -3.25514054e-01,
         1.91018629e-01,  3.11255717e-01, -1.85861168e+00,  2.54959464e-02,
         1.24690866e-01, -4.5

## Save data in Redis

- redis hashes
- keyname = test:register

In [19]:
for record in records:
  name_role = record['name_role']
  vector = record['Facial_Features']

  # convert numpy array into bytes
  vector_bytes = vector.tobytes()

  # save data into redis cloud
  r.hset(name='test:register', key=name_role, value=vector_bytes)

## Retrieve Data from Redis

In [20]:
# Retrieve Records from Redis
retrive_dict = r.hgetall(name='test:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

# Convert binary data to numpy float32 arrays
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))

# Decode the index values
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))
retrive_series.index = index

# Convert series to dataframe and reset index
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role', 'facial_features']
retrive_df

,name_role,facial_features
0,Morgan Freeman@Teacher,"[2.720083e+23, 1.7428625, -0.0, -1.7798909, 2...."
1,Barack Obama@Teacher,"[-107374184.0, 1.862155, -0.0, -1.7645978, 1.5..."
2,Chris Evans@Student,"[-107374184.0, 1.8050444, 0.0, -1.9335388, 0.0..."
3,Angelina Jolie@Student,"[0.0, 1.7161596, -107374184.0, 1.8843765, -0.0..."
4,Scarlett Johansson@Student,"[-107374184.0, 1.5662426, -1.5881868e-23, -1.7..."
